In [345]:
import datetime
import os
import re
import numpy as np
from seabird import fCNV
import seapy
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import matplotlib.colors as colors
import scipy.io as sio
import cmaps
from matplotlib.colors import LinearSegmentedColormap
from cotede import datasets, qctests

In [658]:
class insitu_workspace():
    def __init__(self,MY_ROOT,worksapce_name, workspace_detail=None):
        self.my_root = MY_ROOT
        self.workspace_name = worksapce_name
        self.workspace_detail = workspace_detail
        self.workspace_subdir = os.path.join(self.my_root, self.workspace_name)
        print('Workspace: ' + self.workspace_subdir)
        self.ctd_files = seapy.list_files(os.path.join(self.workspace_subdir+'/CTD/*acfld.cnv'))
        self.ctd_stationinfo = os.path.join(self.workspace_subdir,self.workspace_name+'-station.txt')
        self.isempty = True
        
        
    class Data():
        def __init__(self):
            self.filetype = 'ctd data structure'
            

    def smooth(self,x,window_len=11,window='hanning'):
        """smooth the data using a window with requested size.

        This method is based on the convolution of a scaled window with the signal.
        The signal is prepared by introducing reflected copies of the signal 
        (with the window size) in both ends so that transient parts are minimized
        in the begining and end part of the output signal.

        input:
            x: the input signal 
            window_len: the dimension of the smoothing window; should be an odd integer
            window: the type of window from 'flat', 'hanning', 'hamming', 'bartlett', 'blackman'
                flat window will produce a moving average smoothing.

        output:
            the smoothed signal

        example:

        t=linspace(-2,2,0.1)
        x=sin(t)+randn(len(t))*0.1
        y=smooth(x)

        see also: 

        numpy.hanning, numpy.hamming, numpy.bartlett, numpy.blackman, numpy.convolve
        scipy.signal.lfilter

        TODO: the window parameter could be the window itself if an array instead of a string
        NOTE: length(output) != length(input), to correct this: return y[(window_len/2-1):-(window_len/2)] instead of just y.
        """

    #     if x.ndim != 1:
    #         raise ValueError, "smooth only accepts 1 dimension arrays."

    #     if x.size < window_len:
    #         raise ValueError, "Input vector needs to be bigger than window size."


        if window_len<3:
            return x


        # if not window in ['flat', 'hanning', 'hamming', 'bartlett', 'blackman']:
        #     raise ValueError, "Window is on of 'flat', 'hanning', 'hamming', 'bartlett', 'blackman'"


        s=np.r_[x[window_len-1:0:-1],x,x[-2:-window_len-1:-1]]
        #print(len(s))
        if window == 'flat': #moving average
            w=np.ones(window_len,'d')
        else:
            w=eval('np.'+window+'(window_len)')

        y=np.convolve(w/w.sum(),s,mode='valid')
        return y

    def pull_downcast(self,ctd_file,VAR):
        def lanczos_filter(data,Cf,Nf,M):
            coef = sp.signal.firwin(M+1, Cf/Nf,width = 2/len(data),window='lanczos', pass_zero='lowpass')
            return sp.signal.filtfilt(coef,1.0,data)

        dt = 0.04 #sec
        fs = 1/(dt)
        Nf = fs/2
        Cf = fs/1500
        M = 100
        print(ctd_file)
        profile = fCNV(ctd_file)
        time    = profile['timeS'][:]
        if VAR == 'PSAL2':
            var = profile.data[-6][:]
        else:
            var     = profile[VAR][:]
        pres    = profile['PRES'][:]
        depth   = profile['DEPTH'][:]
        # smooth the noise further
        pres_conv = np.convolve(pres,np.ones(24*6)/24/6,mode='full')[72:-71]
        pres_conv = seapy.filt.bandpass(pres_conv,dt,low_cutoff=dt*200,order=8)
        diff_pres = np.diff(pres_conv)
    #    diff_pres_conv = np.convolve(diff_pres,np.ones(24*6)/24/6,mode='full')[72:-71]
        diff_pres_conv = diff_pres.copy()
        count_down = 0
        count_up = 0
        for i in range(1,len(diff_pres_conv)):
            # print(diff_pres[i])
            # print(diff_pres[i]*diff_pres[i-1])
            if (diff_pres_conv[i]*diff_pres_conv[i-1]) < 0 :
                if diff_pres_conv[i-1]>0:
                    if  not count_down:
                        start_down = i
                    else:
                        start_down = np.r_[start_down, i]
                    count_down+=1
                else:
                    if not count_up :
                        start_up = i
                    else:
                        start_up = np.r_[start_up, i]
                    count_up+=1


    #        print(start_up)
    #        print(start_down)


        # print(pres[start_down[1]])
        larger = 1
        smaller = -1
        equal = 0
    #    print(len(start_down))
    #    print(len(start_up))
        if len(start_up) > len(start_down):
            end_indx = len(start_down)
            status = larger
        elif len(start_up) < len(start_down):
            end_indx = len(start_up)
            status = smaller
        else:
            status = equal


        if start_down[0] < start_up[0]:
            if status == equal:
                mono_duration = start_up[:-1] - start_down[1:]
            elif status == larger:
                mono_duration = start_up[:end_indx-1] - start_down[1:]
            elif status == smaller:
                mono_duration = start_up[:] - start_down[1:]
    #        print(mono_duration)
            downcast_index = np.argmax(np.abs(mono_duration))
    #        print(downcast_index)
            if ((pres[start_down[downcast_index+1]] -
                      pres[start_up[downcast_index]]) < 5):
                mono_duration[downcast_index] = 0
                downcast_index = np.argmax(np.abs(mono_duration))
            mono_duration[downcast_index] = 0

            downcast2_index = np.argmax(np.abs(mono_duration))

            if (downcast2_index > downcast_index)\
                and ((pres[start_down[downcast2_index+1]] - 
                      pres[start_up[downcast2_index]]) > 5):
                time_downcast = np.r_[
                                      time[start_up[downcast_index]:start_down[downcast_index+1]],
                                      time[start_up[downcast2_index]:start_down[downcast2_index+1]]

                                      ]
                pres_downcast = np.r_[
                                      pres[start_up[downcast_index]:start_down[downcast_index+1]],
                                      pres[start_up[downcast2_index]:start_down[downcast2_index+1]]
                                      ]
                depth_downcast = np.r_[
                                      depth[start_up[downcast_index]:start_down[downcast_index+1]],
                                      depth[start_up[downcast2_index]:start_down[downcast2_index+1]]
                                      ]
                var_downcast = np.r_[
                                       var[start_up[downcast_index]:start_down[downcast_index+1]],
                                       var[start_up[downcast2_index]:start_down[downcast2_index+1]]
                                    ]
            elif (downcast2_index < downcast_index)\
                and ((((pres[start_down[downcast2_index+1]] - 
                  pres[start_up[downcast2_index]]) > np.max(pres) *0.4) \
                and ((time[start_up[downcast_index]] -
                  time[start_down[downcast2_index+1]]) <np.max(time)*0.1))\
                or (((pres[start_down[downcast2_index+1]] - 
                  pres[start_up[downcast2_index]]) > np.max(pres) *0.2) \
                and ((time[start_up[downcast_index]] -
                  time[start_down[downcast2_index+1]]) <np.max(time)*0.05))\
                or((time[start_up[downcast_index]] -
                  time[start_down[downcast2_index+1]]) <np.max(time)*0.005)): # judge that the interval between two downcast is small enough
                time_downcast = np.r_[
                                      time[start_up[downcast2_index]:start_down[downcast2_index+1]],
                                      time[start_up[downcast_index]:start_down[downcast_index+1]]

                                      ]
                pres_downcast = np.r_[
                                      pres[start_up[downcast2_index]:start_down[downcast2_index+1]],
                                      pres[start_up[downcast_index]:start_down[downcast_index+1]]
                                      ]
                depth_downcast = np.r_[
                                      depth[start_up[downcast2_index]:start_down[downcast2_index+1]],
                                      depth[start_up[downcast_index]:start_down[downcast_index+1]]
                                      ]
                var_downcast = np.r_[
                                       var[start_up[downcast2_index]:start_down[downcast2_index+1]],
                                       var[start_up[downcast_index]:start_down[downcast_index+1]]
                                    ]
            else:

                time_downcast = time[start_up[downcast_index]:start_down[downcast_index+1]]
                pres_downcast = pres[start_up[downcast_index]:start_down[downcast_index+1]]
                depth_downcast = depth[start_up[downcast_index]:start_down[downcast_index+1]]
                var_downcast = var[start_up[downcast_index]:start_down[downcast_index+1]]
                
            



        else:
            if status == equal:
                mono_duration = start_up - start_down
            elif status == larger:
                mono_duration = start_up[:end_indx] - start_down
            elif status == smaller:
                mono_duration = start_up[:] - start_down[:end_indx]

    #        print(mono_duration)
            downcast_index = np.argmax(np.abs(mono_duration))
    #        print(downcast_index)
            if ((pres[start_down[downcast_index]] -
                      pres[start_up[downcast_index]]) < 5):
                mono_duration[downcast_index] = 0
                downcast_index = np.argmax(np.abs(mono_duration))
            mono_duration[downcast_index] = 0
            downcast2_index = np.argmax(np.abs(mono_duration))
            if (downcast2_index > downcast_index)\
                and ((pres[start_down[downcast2_index]] - 
                      pres[start_up[downcast2_index]]) > 5):
                time_downcast = np.r_[
                                      time[start_up[downcast_index]:start_down[downcast_index]],
                                      time[start_up[downcast2_index]:start_down[downcast2_index]]

                                      ]
                pres_downcast = np.r_[
                                      pres[start_up[downcast_index]:start_down[downcast_index]],
                                      pres[start_up[downcast2_index]:start_down[downcast2_index]]
                                      ]
                depth_downcast = np.r_[
                                      depth[start_up[downcast_index]:start_down[downcast_index]],
                                      depth[start_up[downcast2_index]:start_down[downcast2_index]]
                                      ]
                
                var_downcast = np.r_[
                                       var[start_up[downcast_index]:start_down[downcast_index]],
                                       var[start_up[downcast2_index]:start_down[downcast2_index]]
                                    ]
            elif (downcast2_index < downcast_index)\
                 and ((((pres[start_down[downcast2_index]] - 
                  pres[start_up[downcast2_index]]) > np.max(pres)*0.4) \
                and ((time[start_up[downcast_index]] -
                  time[start_down[downcast2_index]]) <np.max(time)*0.1))\
              or (((pres[start_down[downcast2_index]] - 
                  pres[start_up[downcast2_index]]) > np.max(pres)*0.2) \
                and ((time[start_up[downcast_index]] -
                  time[start_down[downcast2_index]]) <np.max(time)*0.05))\
                or((time[start_up[downcast_index]] -
                  time[start_down[downcast2_index]]) <np.max(time)*0.005)): # judge that the interval between two downcast is small enough
                time_downcast = np.r_[
                                      time[start_up[downcast2_index]:start_down[downcast2_index]],
                                      time[start_up[downcast_index]:start_down[downcast_index]]

                                      ]
                pres_downcast = np.r_[
                                      pres[start_up[downcast2_index]:start_down[downcast2_index]],
                                      pres[start_up[downcast_index]:start_down[downcast_index]]
                                      ]
                depth_downcast = np.r_[
                                      depth[start_up[downcast2_index]:start_down[downcast2_index]],
                                      depth[start_up[downcast_index]:start_down[downcast_index]]
                                      ]
                var_downcast = np.r_[
                                       var[start_up[downcast2_index]:start_down[downcast2_index]],
                                       var[start_up[downcast_index]:start_down[downcast_index]]
                                    ]
            else:

                time_downcast = time[start_up[downcast_index]:start_down[downcast_index]]
                pres_downcast = pres[start_up[downcast_index]:start_down[downcast_index]]
                depth_downcast = depth[start_up[downcast_index]:start_down[downcast_index]]
                var_downcast = var[start_up[downcast_index]:start_down[downcast_index]]
            
            
        return time_downcast.data, pres_downcast.data, depth_downcast.data, var_downcast.data

    def pull_ctdinfos(self):
        self.ctd_infos = {}
        ctdid = open(self.ctd_stationinfo)
        info = ctdid.readline().split()
        cid = 0
        while info:
            self.ctd_infos[info[1]]={}
            self.ctd_infos[info[1]]['TIME'] = (datetime.datetime.strptime(info[2],'%Y%m%d%H%M') - 
                                              datetime.datetime(1970,1,1)).total_seconds()/3600/24
            self.ctd_infos[info[1]]['LONGITUDE']=info[3]
            self.ctd_infos[info[1]]['LATITUDE']=info[4]
            self.ctd_infos[info[1]]['DEPTH'] = info[5]
            self.ctd_infos[info[1]]['CTDID'] = cid
            cid +=1
            info = ctdid.readline().split()
        ctdid.close()
        
    def pull_alldata(self,VARS, interp2meter=0):
        if self.isempty:
            self.data = {}
            self.isempty = False
        for iv, var in enumerate(VARS):
            self.data[var] = self.Data()
            for ic, ctd_file in enumerate(self.ctd_files):
                # print(ctd_file)
                ctdid = re.findall(r"%s/CTD/(.*)acfld.cnv"%(self.workspace_subdir),ctd_file)[0]
                print(ctdid)
                if ic == 0:
                    _, self.data[var].pres, self.data[var].depth, self.data[var].value = self.pull_downcast(ctd_file,var)
                    #remove negative depth
                    valid_depth = np.where(self.data[var].depth>0)
                    self.data[var].pres = self.data[var].pres[valid_depth]
                    self.data[var].depth = self.data[var].depth[valid_depth]
                    self.data[var].value = self.data[var].value[valid_depth]
                    self.data[var].station = np.tile(ctdid,[len(self.data[var].value),]).ravel()
                    self.data[var].time = np.tile(float(self.ctd_infos[ctdid]['TIME']),[len(self.data[var].value),]).ravel()
                    self.data[var].lon = np.tile(float(self.ctd_infos[ctdid]['LONGITUDE']),[len(self.data[var].value),]).ravel()
                    self.data[var].lat = np.tile(float(self.ctd_infos[ctdid]['LATITUDE']),[len(self.data[var].value),]).ravel()
                    if interp2meter:
                        print('interp is activate')
                        
                        data_len = int(np.floor(0.95*len(self.data[var].value)))
                        pres_cut = self.data[var].pres[:data_len] 
                        # splitted based on pressure
                        # cut the tail of data
                        pres_levels = np.arange(np.ceil(np.min(pres_cut)),
                                                   np.floor(np.max(pres_cut)),
                                                   )
                        
                        self.data[var].station_i = np.tile(ctdid,[len(pres_levels),]).ravel()
                        self.data[var].time_i = np.tile(float(self.ctd_infos[ctdid]['TIME']),[len(pres_levels),]).ravel()
                        self.data[var].lon_i = np.tile(float(self.ctd_infos[ctdid]['LONGITUDE']),[len(pres_levels),]).ravel()
                        self.data[var].lat_i = np.tile(float(self.ctd_infos[ctdid]['LATITUDE']),[len(pres_levels),]).ravel()
                        self.data[var].depth_i = seapy.seawater.depth(pres_levels, lat=float(self.ctd_infos[ctdid]['LATITUDE']))
                        self.data[var].pres_i = pres_levels.copy()
                        self.data[var].value_i = np.interp(pres_levels,pres_cut,self.data[var].value[:data_len])
                        
                        
                else:
                    _, thispres, thisdepth, thisvalue = self.pull_downcast(ctd_file,var)
                    #remove negative depth
                    valid_depth = np.where(thisdepth>0)
                    thispres = thispres[valid_depth]
                    thisvalue = thisvalue[valid_depth]
                    thisdepth = thisdepth[valid_depth]
                    thisstation = np.tile(ctdid,[len(thisvalue),]).ravel()
                    thistime = np.tile(float(self.ctd_infos[ctdid]['TIME']),[len(thisvalue),]).ravel()
                    thislon = np.tile(float(self.ctd_infos[ctdid]['LONGITUDE']),[len(thisvalue),]).ravel()
                    thislat = np.tile(float(self.ctd_infos[ctdid]['LATITUDE']),[len(thisvalue),]).ravel()
                    self.data[var].station  = np.r_[self.data[var].station ,thisstation]
                    self.data[var].time  = np.r_[self.data[var].time ,thistime]
                    self.data[var].pres  = np.r_[self.data[var].pres ,thispres]
                    self.data[var].depth = np.r_[self.data[var].depth,thisdepth]
                    self.data[var].value = np.r_[self.data[var].value,thisvalue]
                    self.data[var].lon   = np.r_[self.data[var].lon,thislon]
                    self.data[var].lat   = np.r_[self.data[var].lat,thislat]
                    if interp2meter:
                        print('interp is activate')
                        
                        data_len = int(np.floor(0.95*len(thisvalue)))
                        pres_cut = thispres[:data_len] 
                        # splitted based on pressure
                        # cut the tail of data
                        pres_levels = np.arange(np.ceil(np.min(pres_cut)),
                                                   np.floor(np.max(pres_cut)),
                                                   )
                        
                        thisstation_i = np.tile(ctdid,[len(pres_levels),]).ravel()
                        thistime_i = np.tile(float(self.ctd_infos[ctdid]['TIME']),[len(pres_levels),]).ravel()
                        thislon_i = np.tile(float(self.ctd_infos[ctdid]['LONGITUDE']),[len(pres_levels),]).ravel()
                        thislat_i = np.tile(float(self.ctd_infos[ctdid]['LATITUDE']),[len(pres_levels),]).ravel()
                        thisdepth_i = seapy.seawater.depth(pres_levels, lat=float(self.ctd_infos[ctdid]['LATITUDE']))
                        thispres_i = pres_levels.copy()
                        thisvalue_i = np.interp(pres_levels,pres_cut,thisvalue[:data_len])
                        self.data[var].station_i = np.r_[self.data[var].station_i, thisstation_i]
                        self.data[var].time_i = np.r_[self.data[var].time_i, thistime_i]
                        self.data[var].lon_i = np.r_[self.data[var].lon_i, thislon_i]
                        self.data[var].lat_i = np.r_[self.data[var].lat_i, thislat_i]
                        self.data[var].depth_i = np.r_[self.data[var].depth_i, thisdepth_i]
                        self.data[var].pres_i = np.r_[self.data[var].pres_i, thispres_i]
                        self.data[var].value_i = np.r_[self.data[var].value_i,thisvalue_i]
                    
    
        
            
                    
                    
                   
                
                
            
        

In [659]:
test =insitu_workspace('/Volumes/TO_1/roms4dvar_ecs/i4dvar_outputs/INSITU_OBS','NSFC2012SPRING-ES','NSFC2012SPRING-ES')

Workspace: /Volumes/TO_1/roms4dvar_ecs/i4dvar_outputs/INSITU_OBS/NSFC2012SPRING-ES


In [660]:
test.pull_ctdinfos()

In [661]:
# must pull_ctdinfos before pull_alldata
test.pull_alldata(VARS=['TEMP','PSAL2'],interp2meter=True)

DH3-01
/Volumes/TO_1/roms4dvar_ecs/i4dvar_outputs/INSITU_OBS/NSFC2012SPRING-ES/CTD/DH3-01acfld.cnv
interp is activate
DH3-02
/Volumes/TO_1/roms4dvar_ecs/i4dvar_outputs/INSITU_OBS/NSFC2012SPRING-ES/CTD/DH3-02acfld.cnv
interp is activate
DH3-03
/Volumes/TO_1/roms4dvar_ecs/i4dvar_outputs/INSITU_OBS/NSFC2012SPRING-ES/CTD/DH3-03acfld.cnv
interp is activate
DH3-04
/Volumes/TO_1/roms4dvar_ecs/i4dvar_outputs/INSITU_OBS/NSFC2012SPRING-ES/CTD/DH3-04acfld.cnv
interp is activate
DH3-05
/Volumes/TO_1/roms4dvar_ecs/i4dvar_outputs/INSITU_OBS/NSFC2012SPRING-ES/CTD/DH3-05acfld.cnv
interp is activate
DH3-06
/Volumes/TO_1/roms4dvar_ecs/i4dvar_outputs/INSITU_OBS/NSFC2012SPRING-ES/CTD/DH3-06acfld.cnv
interp is activate
DH3-07
/Volumes/TO_1/roms4dvar_ecs/i4dvar_outputs/INSITU_OBS/NSFC2012SPRING-ES/CTD/DH3-07acfld.cnv
interp is activate
DH3-08
/Volumes/TO_1/roms4dvar_ecs/i4dvar_outputs/INSITU_OBS/NSFC2012SPRING-ES/CTD/DH3-08acfld.cnv
interp is activate
DH4-01
/Volumes/TO_1/roms4dvar_ecs/i4dvar_outputs/INSITU

In [601]:
np.unique(test.data['TEMP'].station[station_position])

array(['DH7-01', 'DH7-02', 'DH7-03', 'DH7-04', 'DH7-05', 'ZD-21', 'ZD-24'],
      dtype='<U6')

In [615]:
plt.plot(test.data['TEMP'].lat[station_position])

In [664]:
#temp profile
%matplotlib
import matplotlib.tri as tri
# VAR='PSAL2'
VAR='TEMP'
# -------------NSFC2012SPRING-YS
# stations = ['E01','E02','E03','E04','E05']
# stations = ['H01','H02','H03','H04','H05','H06','H07','H08']
# stations = ['H38','H39','H40','H41','H42']
# stations = ['H33','H34','H35','H36']
# stations = ['H25','H26','H27','H28','H29']
# stations = ['H10','H11','H12','H13','H14','H15','H16','H17','H18','H19']
# stations = ['HF1','HF2','HF3','H43','H44']
# -------------NSFC2012SPRING-ES
# stations = ['DH3-01','RB-10','RB-16','DH3-02','DH3-03','DH3-04','DH3-05','DH3-06','DH3-07','DH3-08',]
# stations = ['DH4-01','DH4-02','DH4-03','DH4-04','DH4-05','DH4-06','ZA-05','ZA-03','ZA-01']
# stations = ['GC-05','ZB-07','ZB-09','ZB-11','DH5-01','GC-06','ZB-12','DH5-02','DH5-03','DH5-04','DH5-05','DH5-06']
# stations = ['DH6-05','DH6-04','DH6-03','DH6-02','ZC-17','DH6-01','ZC-15','ZC-13']
# stations = ['ZD-19','ZD-21','DH7-01','ZD-24','DH7-02','DH7-03','DH7-04','DH7-05']
# stations = ['DH8-05','DH8-04','DH8-03','DH8-02','ZE-27','DH8-01','ZE-25']
stations = ['GC-07', 'DH9-01','DH9-02','DH9-03','DH9-04','DH9-05','DH9-06']
fig, ax = plt.subplots()
i = 0
for istation in stations:
    # print(istation)
    # print(test.ctd_infos[istation]['CTDID'])
    current_position = np.where(test.data[VAR].station == istation)[0]
    # print(current_position)
    if i == 0:
        station_position = current_position.copy()
        
    else:
        station_position = np.r_[station_position,current_position]
        
    i+=1
    
lon_station = test.data[VAR].lon[station_position]
lat_station = test.data[VAR].lat[station_position]
depth_station = test.data[VAR].depth[station_position]
data_station = test.data[VAR].value[station_position]
cntr = ax.tricontourf(lon_station,depth_station,data_station,levels=30,cmap='jet')

ax.scatter(np.unique(lon_station),np.zeros_like(np.unique(lon_station)), color='k')
ax.invert_yaxis()

fig.colorbar(cntr, ax=ax)

Using matplotlib backend: MacOSX


In [665]:
fig, ax = plt.subplots()
i = 0
for istation in stations:
    # print(istation)
    # print(test.ctd_infos[istation]['CTDID'])
    current_position = np.where(test.data[VAR].station_i == istation)[0]
    # print(current_position)
    if i == 0:
        station_position = current_position.copy()
        
    else:
        station_position = np.r_[station_position,current_position]
        
    i+=1
    
lon_station = test.data[VAR].lon_i[station_position]
lat_station = test.data[VAR].lat_i[station_position]
depth_station = test.data[VAR].depth_i[station_position]
data_station = test.data[VAR].value_i[station_position]
cntr = ax.tricontourf(lon_station,depth_station,data_station,levels=30,cmap='jet')

ax.scatter(np.unique(lon_station),np.zeros_like(np.unique(lon_station)),color='k')
ax.invert_yaxis()
fig.colorbar(cntr, ax=ax)

2024-12-22 23:42:49.349 python[27713:2136446] error messaging the mach port for IMKCFRunLoopWakeUpReliable


In [657]:
pres_station

array([ -1.073,  -1.07 ,  -1.068, ..., 106.026, 106.019, 106.013])

In [656]:
station = 'DH9-06'
fig, ax = plt.subplots()
current_position = np.where(test.data[VAR].station == station)[0]
    # print(current_position)

station_position = current_position.copy()


    
lon_station = test.data[VAR].lon[station_position]
lat_station = test.data[VAR].lat[station_position]
pres_station = test.data[VAR].pres[station_position]
depth_station = test.data[VAR].depth[station_position]
data_station = test.data[VAR].value[station_position]
ax.plot(data_station,depth_station)
ax.invert_yaxis()

In [626]:
station = 'DH9-06'
fig, ax = plt.subplots()
current_position = np.where(test.data[VAR].station_i == station)[0]
    # print(current_position)

station_position = current_position.copy()


    
lon_station = test.data[VAR].lon_i[station_position]
lat_station = test.data[VAR].lat_i[station_position]
depth_station = test.data[VAR].depth_i[station_position]
data_station = test.data[VAR].value_i[station_position]
ax.plot(data_station,depth_station)
ax.invert_yaxis()

In [87]:
position = np.where(test.data['TEMP'].time==test.data['TEMP'].time[1])

In [91]:
seapy.seawater.depth(test.data['TEMP'].pres[position],test.data['TEMP'].lat[position])

array([ 1.6733532 ,  1.65547769,  1.64058143, ..., 14.99712619,
       14.98719599, 14.9772658 ])

In [92]:
test.data['TEMP'].depth[position]

array([ 1.673,  1.655,  1.641, ..., 14.997, 14.987, 14.977])

In [88]:
position

(array([   0,    1,    2, ..., 1231, 1232, 1233]),)

In [170]:
test.ctd_infos

{'E01': {'TIME': 15462.325694444444,
  'LONGITUDE': '122.4246',
  'LATITUDE': '31.2741',
  'DEPTH': '15.2400',
  'CTDID': 0},
 'E02': {'TIME': 15462.529861111112,
  'LONGITUDE': '123.0984',
  'LATITUDE': '31.2746',
  'DEPTH': '54.2400',
  'CTDID': 1},
 'E03': {'TIME': 15462.758333333333,
  'LONGITUDE': '123.9099',
  'LATITUDE': '31.2771',
  'DEPTH': '50.7500',
  'CTDID': 2},
 'E04': {'TIME': 15462.924305555556,
  'LONGITUDE': '124.5479',
  'LATITUDE': '31.2739',
  'DEPTH': '50.5900',
  'CTDID': 3},
 'E05': {'TIME': 15463.111111111111,
  'LONGITUDE': '125.3320',
  'LATITUDE': '31.2730',
  'DEPTH': '53.8100',
  'CTDID': 4},
 'H01': {'TIME': 15469.810416666667,
  'LONGITUDE': '121.0122',
  'LATITUDE': '35.9648',
  'DEPTH': '31.8900',
  'CTDID': 5},
 'H02': {'TIME': 15469.711805555555,
  'LONGITUDE': '121.3415',
  'LATITUDE': '35.9656',
  'DEPTH': '37.1700',
  'CTDID': 6},
 'H03': {'TIME': 15469.622222222222,
  'LONGITUDE': '121.6709',
  'LATITUDE': '35.9686',
  'DEPTH': '35.1200',
  'CTDI

In [54]:
test.ctd_files

['/Volumes/TO_1/roms4dvar_ecs/i4dvar_outputs/INSITU_OBS/NSFC2012SPRING-YS/CTD/E01acfld.cnv',
 '/Volumes/TO_1/roms4dvar_ecs/i4dvar_outputs/INSITU_OBS/NSFC2012SPRING-YS/CTD/E02acfld.cnv',
 '/Volumes/TO_1/roms4dvar_ecs/i4dvar_outputs/INSITU_OBS/NSFC2012SPRING-YS/CTD/E03acfld.cnv',
 '/Volumes/TO_1/roms4dvar_ecs/i4dvar_outputs/INSITU_OBS/NSFC2012SPRING-YS/CTD/E04acfld.cnv',
 '/Volumes/TO_1/roms4dvar_ecs/i4dvar_outputs/INSITU_OBS/NSFC2012SPRING-YS/CTD/E05acfld.cnv',
 '/Volumes/TO_1/roms4dvar_ecs/i4dvar_outputs/INSITU_OBS/NSFC2012SPRING-YS/CTD/H01acfld.cnv',
 '/Volumes/TO_1/roms4dvar_ecs/i4dvar_outputs/INSITU_OBS/NSFC2012SPRING-YS/CTD/H02acfld.cnv',
 '/Volumes/TO_1/roms4dvar_ecs/i4dvar_outputs/INSITU_OBS/NSFC2012SPRING-YS/CTD/H03acfld.cnv',
 '/Volumes/TO_1/roms4dvar_ecs/i4dvar_outputs/INSITU_OBS/NSFC2012SPRING-YS/CTD/H04acfld.cnv',
 '/Volumes/TO_1/roms4dvar_ecs/i4dvar_outputs/INSITU_OBS/NSFC2012SPRING-YS/CTD/H05acfld.cnv',
 '/Volumes/TO_1/roms4dvar_ecs/i4dvar_outputs/INSITU_OBS/NSFC2012SPRING

In [67]:
sst_obs = seapy.roms.obs.obs(filename='/Volumes/TO_1/roms4dvar_ecs/i4dvar_outputs/NOAA_GHRSST/geopolar_sst_201205.nc')

In [61]:
# merge sst and in-situ togethor
grdfile = '/Volumes/TO_1/outputs_SCORRECTION/ocean_ecs_fwd_15461_outer1.nc'
dt = 1/24
# reftime = datetime.datetime(1970,1,1)


In [13]:
grid = seapy.model.asgrid(grdfile)

In [63]:
data = [seapy.roms.obs.raw_data("TEMP", "CTD", test.data['TEMP'].value,None,0.1)]
        
insitu_obs = seapy.roms.obs.gridder(grid,test.data['TEMP'].time,
                                    test.data['TEMP'].lon,test.data['TEMP'].lat,
                                    -test.data['TEMP'].depth, data,dt,title="cruise obs")

Output()

In [68]:
sst_obs.add(insitu_obs)

In [69]:
sst_obs.to_netcdf('./combined_sst.nc')

In [66]:
pwd

'/Volumes/TO_1/roms4dvar_ecs/i4dvar_analysis/obs_processing'

array([371095.81666667, 371095.81666667, 371095.81666667, ...,
       371246.7       , 371246.7       , 371246.7       ])